In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
from math import sqrt

In [ ]:
df_train=pd.read_csv('../input/bike-sharing-demand/train.csv')
df_test=pd.read_csv('../input/bike-sharing-demand/test.csv')

In [ ]:
df_test.head()

In [ ]:
df_train.tail()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
#df_train.isnull().sum()
df_test.isnull().sum()

In [ ]:
# tr=df_train.duplicated().sum()
# tr
ts=df_train.duplicated().sum()
ts

In [ ]:
columns=df_train.columns
columns

In [ ]:
for i in columns:
    print(df_train[str(i)].value_counts())
    print('**********************************')

In [ ]:
df_train1=pd.DataFrame(df_train.corr().unstack().sort_values(ascending=False)['count'],columns=['Correlation to the target'])
df_train1.style.background_gradient(cmap=sns.light_palette("red", as_cmap=True))

In [ ]:
plt.figure(figsize=(10,6))
sns.distplot(df_train['count'], color='r')
#df_train['count'].hist(bins=50)
plt.show()

In [ ]:

#df_train['minute'] = df_train['datetime'].dt.minute


In [ ]:
#conclusion that 4 season with count have close proportions 
df_train[df_train['season']==1]['count'].hist(bins=20) #blue
df_train[df_train['season']==2]['count'].hist() #orange
df_train[df_train['season']==3]['count'].hist() #green
df_train[df_train['season']==4]['count'].hist() #red
plt.legend(['spring', 'summer', 'fall',  'winter' ])
plt.show()

In [ ]:
df_train.plot(x='datetime',y='count',kind='line')
plt.show()

In [ ]:
test_date=pd.to_datetime(df_test['datetime'])

In [ ]:
type(test_date)

In [ ]:
def func_feature(df):
    df['datetime']= pd.to_datetime(df['datetime'])
    df['year'] = df['datetime'].dt.year
    df['month'] = df['datetime'].dt.month
    df['day'] = df['datetime'].dt.day
    df['hour'] = df['datetime'].dt.hour
    
    df['weather'] = df.weather.map({1:'Spring', 2:'Summer', 3:'Fall', 4:'Winter'})
    df['season'] = df.season.map({1:'Clear', 2:'Mist', 3:'Light rain', 4:'Heavy rain'})
    features = ['season', 'weather']
    for i in features:
        df[i] = df[i].astype('category')

    df = pd.get_dummies(df,drop_first=True)  
    df.drop('datetime', axis = 1, inplace = True)
    
    return df

In [ ]:
df_train=func_feature(df_train)
df_test=func_feature(df_test)

In [ ]:
df_train.head()

In [ ]:
df_train1=pd.DataFrame(df_train.corr().unstack().sort_values(ascending=False)['count'],columns=['Correlation to the target'])
df_train1.style.background_gradient(cmap=sns.light_palette("red", as_cmap=True))

In [ ]:
#
df_train.info()

In [ ]:
df_train.plot(x='year',y='count',kind='bar')
plt.show()

In [ ]:
df_train.plot(x='month',y='count',kind='bar')
plt.show()

In [ ]:
df_train.plot(x='day',y='count',kind='bar')
plt.show()

In [ ]:
#we have two peaks after 5 am :10am ,and after 15 pm :20pm
df_train.plot(x='hour',y='count',kind='bar')
plt.show()

In [ ]:
#registered  and casual not in test data so i need to drop it
X = df_train.drop('count', axis = 1, inplace = False)
X = X.drop('casual', axis = 1, inplace = False)
X = X.drop('registered', axis = 1, inplace = False)
y = df_train['count']

In [ ]:
X.head()

In [ ]:


x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 0)



In [ ]:
print(x_train.shape,y_train.shape)


In [ ]:


rf_reg = RandomForestRegressor(n_estimators = 1000, n_jobs = -1, verbose = 1)
rf_reg.fit(x_train, y_train)
pred1 = rf_reg.predict(x_val)

In [ ]:
print("RMSLE",sqrt(mean_squared_log_error(y_val,pred1)))

In [ ]:
rf_reg.fit(X, y)
prediction = rf_reg.predict(df_test)

In [ ]:
submission = pd.DataFrame({'datetime':test_date, 'count':[max(0, x) for x in prediction]})
submission['count'] = np.round(submission['count']).astype('int')
submission.to_csv('bike_shareing.csv', index = False)